#1. Tunning de parametros para rede LSTM
##2. especificações do tunning

Camadas ocultas|Neuronios|Taxa de aprendizagem|Função de aprendizagem| Epocas
---------------|---------|--------------------|---------|---------|
        1      |  10-x+5-100   | 0.001 - 0.01       |Relu| 10-x+10- 100      
  
        
##3. importar as bibliotecas

###3.1 instalar a biblioteca do keras prórpia para o tunning



In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 2.7 MB/s 


###3.2 importar todas as bibliotecas que serão usadas no processo

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt

##4. Importar os dataset e deixar o dataset na estrutura de entrada

In [ ]:
df_teste = pd.read_parquet('/content/drive/MyDrive/df_teste_norm')
df_treino = pd.read_parquet('/content/drive/MyDrive/df_treino_norm')
df_treino = df_treino.dropna()
df_teste = df_teste.dropna()
df_treino2 = df_treino[['TEMP_NORM','BATIMETRIA_NORM']]
df_teste2 = df_teste[['TEMP_NORM','BATIMETRIA_NORM']]
lista = list()
for x,i in df_treino2.values:
  lista.append([x,i])
sal_treino = np.array(df_treino.SALINIDADE)
pr_treino = np.array(lista)
lista = list()
for x,i in df_teste2.values:
  lista.append([x,i])
sal_teste = np.array(df_teste.SALINIDADE)
pr_teste = np.array(lista)
pr_teste = pr_teste.reshape(pr_teste.shape[0],pr_teste.shape[1],1)
pr_treino = pr_treino.reshape(pr_treino.shape[0],pr_treino.shape[1],1)

## 5. Fazer o tunning

In [ ]:
def KerasTunnerGRU(hp):
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  hiper_param = hp.Int('units', min_value=10, max_value=100, step=5)
  GRU = tf.keras.layers.GRU(units=hiper_param,activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(GRU)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  taxa_apren =  hp.Choice('learning_rate', values=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01])
  adan = tf.keras.optimizers.Adam(learning_rate=taxa_apren)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

tuner = kt.Hyperband(KerasTunnerGRU,
                     objective='val_mae',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir2',
                     project_name='salinidade')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(pr_teste, sal_teste,epochs=10,batch_size=120,validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 03m 23s]
val_mae: 0.867779552936554

Best val_mae So Far: 0.867779552936554
Total elapsed time: 00h 37m 30s
INFO:tensorflow:Oracle triggered exit


###5.1 Resultado

In [ ]:
best_hps.values

{'learning_rate': 0.001,
 'tuner/bracket': 0,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units': 85}

##6. Melhor época para os dados de teste

###6.1 criar função

In [ ]:
def KerasTunnerGRU():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  GRU = tf.keras.layers.GRU(units=best_hps['units'],activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(GRU)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=best_hps['learning_rate'])
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

###6.2 treinar o modelo com os dados de teste para a melhor epoca

In [ ]:
modelo  = KerasTunnerGRU()
best_epoch = modelo.fit(pr_treino,sal_treino,epochs=100,batch_size=120)
val_acc_per_epoch = best_epoch.history['mae']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch))+1
modelo.fit(pr_treino,sal_treino,epochs=best_epoch,batch_size=120)
modelo.evaluate(pr_teste,sal_teste)

Epoch 1/100
10093/10093 [==============================] - 47s 5ms/step - loss: 16.0800 - mae: 1.3300 - mse: 16.0800
Epoch 2/100
10093/10093 [==============================] - 47s 5ms/step - loss: 4.5029 - mae: 0.9424 - mse: 4.5029
Epoch 3/100
10093/10093 [==============================] - 47s 5ms/step - loss: 4.4539 - mae: 0.9336 - mse: 4.4539
Epoch 4/100
10093/10093 [==============================] - 46s 5ms/step - loss: 4.4119 - mae: 0.9229 - mse: 4.4119
Epoch 5/100
10093/10093 [==============================] - 46s 5ms/step - loss: 4.3934 - mae: 0.9174 - mse: 4.3934
Epoch 6/100
10093/10093 [==============================] - 46s 5ms/step - loss: 4.3854 - mae: 0.9147 - mse: 4.3854
Epoch 7/100
10093/10093 [==============================] - 47s 5ms/step - loss: 4.3697 - mae: 0.9106 - mse: 4.3697
Epoch 8/100
10093/10093 [==============================] - 48s 5ms/step - loss: 4.3668 - mae: 0.9096 - mse: 4.3668
Epoch 9/100
10093/10093 [==============================] - 48s 5ms/step - loss

[4.300154685974121, 0.864061713218689, 4.300154685974121]

# Resultado
MAE: 0.864061713218689 

MSE: 4.300154685974121

TAXA DE APRENDIZAGEM: 0.001

EPOCH: 99

NEURONIOS: 85